In [1]:
import xarray as xr
import numpy as np

In [13]:
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from open4dense import give_me_orig_ecco
# import oceanspy as ospy
import seaduck as sd
import xarray as xr
from seaduck.get_masks import which_not_stuck
import os 
# import sys
# int_arg = int(sys.argv[-1])
# print('got sys parameter:',int_arg)
int_arg = 0

filedb_lst = []
for i in range(1,13):
    for j in range(1,4):
        filedb_lst.append(f'/sciserver/filedb{i:02}-0{j}')

seed = 2011

save_path = filedb_lst[9+int_arg]+'/ocean/wenrui_temp/particle_file/WA/nc_new/'
path = '/sciserver/filedb02-02/ocean/wenrui_temp/heat/'

ds = give_me_orig_ecco()
ds['utrans'] = (ds['u_gm']+ds['UVELMASS'])*ds.dyG*ds.drF
ds['vtrans'] = (ds['v_gm']+ds['VVELMASS'])*ds.dxG*ds.drF
ds['wtrans'] = (ds['w_gm']+ds['WVELMASS'])*ds.rA
tseas1 = xr.open_zarr(path+'tseas1.zarr')
tseas2 = xr.open_zarr(path+'tseas2.zarr')
tseas3 = xr.open_zarr(path+'tseas3.zarr')
tseas = xr.concat([tseas1,tseas2,tseas3],dim = 'dayofyear')
ta = (ds.THETA.groupby('time.dayofyear') - tseas).transpose('time','Z','face','Y','X').THETA

tub = sd.OceData(ds)

time = '2011-03-01'
t = sd.utils.convert_time(time)
end_time = t-365*86400*1.1
stops = np.array([end_time])

ylim = (-35,-22.5)
xlim = (100,120)

xbool = np.logical_and(ds.XC>xlim[0],ds.XC<xlim[1])
ybool = np.logical_and(ds.YC>ylim[0],ds.YC<ylim[1])
zbool = ds.Z>-50
pos_bool = np.logical_and(np.logical_and(xbool,ybool),zbool)
warm_bool = ta.sel(time = time)[0]>1.5
those = np.logical_and(warm_bool,pos_bool)

p = sd.Particle(
    bool_array=those, num=10000, random_seed=seed,
    t = t,
    data = tub, free_surface = 'noflux',
    save_raw = True,
    uname = 'utrans',vname = 'vtrans',wname = 'wtrans',
    transport  = True
)
p=p.subset(sd.get_masks.which_not_stuck(p))

/home/wjiang33/miniconda3/envs/bubblebath/lib/python3.12/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 26 times more chunks
  return self.array[key]


In [14]:
total_volume_heat = float((tub.Vol*those).sum())/1e9
print(total_volume_dic)

36326.921469952


In [17]:
p.N

10009

In [19]:
volume_each_heat = total_volume_heat/p.N
print(volume_each_heat)

3.6294256638976923


In [23]:
ds = xr.open_zarr('/sciserver/filedb02-02/ocean/wenrui_temp/dic_exp/bgc_tutorial.zarr')
ds['utrans'] = ds['UVELMASS']*ds['drF']*ds['dyG']
ds['vtrans'] = ds['VVELMASS']*ds['drF']*ds['dxG']
ds['wtrans'] = ds['WVELMASS']*ds['rA']
wmean = ds.WVELMASS.mean(dim = 'time').persist()
wtemp = np.zeros((16,64,128))
wtemp[:15] = wmean.data
vert = (wtemp[1:]+wtemp[:-1])/2
vert = xr.DataArray(vert,dims = ('Z','Y','X'))
ymask = np.logical_and(ds.Y>-90,ds.Y<-45)
zmask = np.logical_and(ds.Z>-1080,ds.Z<0)
smask = np.logical_and(vert>0,True)#np.logical_or(rhom<27.1,rhom>27.5))
# smask = np.logical_and(vert>0,True)
mask = np.logical_and(np.logical_and(smask,ymask),zmask)
tub = sd.OceData(ds)
tf = float(ds.time[0])
t0 = float(ds.time[-1])
pt = sd.Particle(
    bool_array=mask, num=10000, random_seed=712,
    t = tf,
    data = tub, free_surface = 'kick_back',
    save_raw = True,
    uname = 'utrans',vname = 'vtrans',wname = 'wtrans',
    transport = True
)

In [24]:
total_volume_heat = float((tub.Vol*mask).sum())/1e9
print(total_volume_heat)

40437212.22700438


In [26]:
pt.N

10196

In [27]:
total_volume_heat/pt.N

3965.98786063205

In [29]:
pre = 'precalc/dic_'
for name in ['lw','iw','dw','bw']:
    mean = xr.open_zarr(pre+name+'mean')
    print(float(mean['negbio'][-1]))

-0.006311798550643893
-0.0029345369271876025
-0.0008244684332516168
-0.0002864631279728903
